In [151]:
import math
import pandas as pd
import numpy as np
import os
import random

In [117]:
data_dir = os.path.join(os.path.abspath('.'), 'hw01', 'input_files')
print('File location -->',data_dir)

data = pd.read_csv(os.path.join(data_dir, 'mushroom_data.txt'), sep=' ', header=None)
data.head(10)

File location --> C:\Users\mmaegaard\hw01\input_files


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,x,s,n,t,p,f,c,n,k,e,...,w,w,p,w,o,p,k,s,u,p
1,x,s,y,t,a,f,c,b,k,e,...,w,w,p,w,o,p,n,n,g,e
2,b,s,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,n,m,e
3,x,y,w,t,p,f,c,n,n,e,...,w,w,p,w,o,p,k,s,u,p
4,x,s,g,f,n,f,w,b,k,t,...,w,w,p,w,o,e,n,a,g,e
5,x,y,y,t,a,f,c,b,n,e,...,w,w,p,w,o,p,k,n,g,e
6,b,s,w,t,a,f,c,b,g,e,...,w,w,p,w,o,p,k,n,m,e
7,b,y,w,t,l,f,c,b,n,e,...,w,w,p,w,o,p,n,s,m,e
8,x,y,w,t,p,f,c,n,p,e,...,w,w,p,w,o,p,k,v,g,p
9,b,s,y,t,a,f,c,b,g,e,...,w,w,p,w,o,p,k,s,m,e


In [119]:
# we can set our parameters like this:
data.goal = data.shape[1]-1 # should return the last column
data.attrs = list(range(len(data.values[0]))) # 'attributes' aka column numbers

In [121]:
data.attrnames = data.attrs # names of columns -- for our case, just numbers
data.examples = data.values # data.examples[0] should return the first row
data.inputs = [a for a in data.attrs if a != data.goal] # should return attributes w/out the goal

In [123]:
data.vals = list(map(lambda x: list(set(x)), zip(*data.examples))) # should hold all values per specified column

In [125]:
print('attributes:',data.attrs)
print('attrnames:',data.attrnames)
print('goal:',data.goal)
print('inputs:',data.inputs)

attributes: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
attrnames: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
goal: 22
inputs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [126]:
data.examples

array([['x', 's', 'n', ..., 's', 'u', 'p'],
       ['x', 's', 'y', ..., 'n', 'g', 'e'],
       ['b', 's', 'w', ..., 'n', 'm', 'e'],
       ...,
       ['x', 'y', 'g', ..., 'y', 'p', 'e'],
       ['x', 'y', 'c', ..., 'c', 'd', 'p'],
       ['f', 'y', 'c', ..., 'c', 'd', 'p']], dtype=object)

In [127]:
data.vals

[['c', 'x', 's', 'k', 'b', 'f'],
 ['g', 'f', 's', 'y'],
 ['w', 'g', 'c', 'n', 'e', 'b', 'p', 'y'],
 ['t', 'f'],
 ['a', 'c', 'l', 'n', 'p', 'm', 'f'],
 ['a', 'f'],
 ['w', 'c'],
 ['n', 'b'],
 ['w', 'g', 'k', 'h', 'r', 'n', 'p', 'u', 'y'],
 ['e', 't'],
 ['c', 'e', 'b', 'r'],
 ['k', 'f', 's', 'y'],
 ['k', 'f', 's', 'y'],
 ['w', 'g', 'c', 'n', 'b', 'p', 'y'],
 ['w', 'g', 'c', 'n', 'b', 'p', 'y'],
 ['p'],
 ['w', 'y'],
 ['o', 'n', 't'],
 ['l', 'e', 'n', 'p'],
 ['w', 'k', 'r', 'h', 'n', 'u'],
 ['a', 'c', 's', 'n', 'v', 'y'],
 ['g', 'l', 'd', 'p', 'm', 'u'],
 ['e', 'p']]

In [128]:
class DecisionSplit:
    def __init__(self, attr, attrname=None, default_child=None, branches=None):
        """Initialize by saying what attribute this node tests."""
        self.attr = attr
        self.attrname = attrname or attr
        self.default_child = default_child
        self.branches = branches or {}

    def __call__(self, example):
        """Given an example, classify it using the attribute and the branches."""
        attrvalue = example[self.attr]
        if attrvalue in self.branches:
            return self.branches[attrvalue](example)
        else:
            # return default class when attribute is unknown
            return self.default_child(example)

    def add(self, val, subtree):
        """Add a branch.  If self.attr = val, go to the given subtree."""
        self.branches[val] = subtree

    def display(self, indent=0):
        name = self.attrname
        print('Test', name)
        for (val, subtree) in self.branches.items():
            print(' ' * 4 * indent, name, '=', val, '==>', end=' ')
            subtree.display(indent + 1)
        print()   # newline

    def __repr__(self):
        return ('DecisionFork({0!r}, {1!r}, {2!r})'
                .format(self.attr, self.attrname, self.branches))

In [129]:
class DecisionLeaf:
    """A leaf of a decision tree holds just a result."""

    def __init__(self, result):
        self.result = result

    def __call__(self, example):
        return self.result

    def display(self, indent=0):
        print('RESULT =', self.result)

    def __repr__(self):
        return repr(self.result)

In [139]:
def argmin_random_tie(seq, fn):
    best_score = fn(seq[0]); n = 0
    for x in seq:
        x_score = fn(x)
        if x_score < best_score:
            best, best_score = x, x_score; n = 1
        elif x_score == best_score:
            n += 1
            if random.randrange(n) == 0:
                best = x
    return best

In [140]:
def argmax_random_tie(seq, fn):
    return argmin_random_tie(seq, lambda x: -fn(x))

In [152]:
def decision_tree_learning(examples, attrs, parent_examples=()):
    if len(examples) == 0:
        return plurality_value(parent_examples)
    elif all_same_class(examples):
        return DecisionLeaf(examples[0][data.goal])
    elif len(attrs) == 0:
        return plurality_value(examples)
    else:
        A = choose_attribute(attrs, examples)
        tree = DecisionSplit(A, data.attrnames[A], plurality_value(examples))
        for (v_k, exs) in split_by(A, examples):
            subset = [x for x in attrs if x != A]
            subtree = decision_tree_learning(exs, subset, examples)
            tree.add(v_k, subtree)
        return tree

def plurality_value(examples):
    popular = argmax_random_tie(data.vals[data.goal],
                                lambda v: count(data.goal, v, examples))
    return DecisionLeaf(popular)

def count(attr, val, examples):
    return sum(e[attr] == val for e in examples)

def all_same_class(examples):
    class0 = examples[0][data.goal]
    return all(e[data.goal] == class0 for e in examples)

def choose_attribute(attrs, examples):
    return argmax_random_tie(attrs,
                             lambda a: information_gain(a, examples))

def information_gain(attr, examples):
    def I(examples):
        return information_content([count(data.goal, v, examples)
                                        for v in data.vals[data.goal]])
    N = len(examples)
    remainder = sum((len(examples_i)/N) * I(examples_i)
                    for (v, examples_i) in split_by(attr, examples))
    return I(examples) - remainder

def split_by(attr, examples):
    return [(v, [e for e in examples if e[attr] == v])
            for v in values[attr]]

def information_content(values):
    clean_val = [x for x in values if x!= 0]
    k = 1.0 / sum(clean_val)
    probs = [k * n for n in clean_val]
    
    return sum(-p * math.log2(p) for p in probs)

In [144]:
data.vals[data.goal]

['e', 'p']

In [153]:
trial = decision_tree_learning(data.examples, data.inputs)

ZeroDivisionError: float division by zero